In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

/Users/plin/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
'''
Read in the data in the file "train.csv".
Write a function that calculate the RMSLE (root mean squared log error) for evaluation of models.
This loss function is more sensitive to the ratio of predicted values to the actual values.
'''
df = pd.read_csv('data/train.csv')
print len(df)
df.head()

/Users/plin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


401125


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def score2(target, predictions):
    log_diff = np.log(predictions + 1) - np.log(target + 1)
    return np.sqrt(np.mean(log_diff ** 2))

In [4]:
'''
Drop the null values and build a naive model with a single feature.
'''
dfclean = df.dropna(subset = ['YearMade', 'Enclosure'])

y = dfclean['SalePrice'].values.reshape(-1, 1)
X = dfclean[['YearMade']].values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

linear = LinearRegression()
linear.fit(X_train, y_train)

test_predicted = linear.predict(X_test)
print 'RMSLE with 1 feature "YearMade":', score2(y_test, test_predicted)

RMSLE with 1 feature "YearMade": 0.721388527347


In [5]:
print len(dfclean)
dfclean.head()

400800


,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
'''
Add another feature "Enclosure" which contains several types of operation systems.
Dummify it, merge it into the original table, and update the score.
'''
dummies = pd.get_dummies(dfclean['Enclosure'])
dummies.head()

,EROPS,EROPS AC,EROPS w AC,NO ROPS,None or Unspecified,OROPS
0,0,0,1,0,0,0
1,0,0,1,0,0,0
2,0,0,0,0,0,1
3,0,0,1,0,0,0
4,1,0,0,0,0,0


In [7]:
dfenclo = pd.concat([dfclean, dummies], axis = 1)
dfenclo = dfenclo.drop(['EROPS AC', 'NO ROPS', 'None or Unspecified'], axis = 1) #dropped because of scarcity
dfenclo.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,EROPS,EROPS w AC,OROPS
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,0,1,0
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,0,1,0
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0


In [8]:
y = dfenclo['SalePrice'].values.reshape(-1, 1)
X = dfenclo[['YearMade', 'EROPS', 'EROPS w AC']].values.reshape(-1, 3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

linear = LinearRegression()
linear.fit(X_train, y_train)

test_predicted = linear.predict(X_test)
print 'RMSLE with 2 features "YearMade" and "Enclosure":', score2(y_test, test_predicted)
#a better model with lower error when additional feature is included!

RMSLE with 2 features "YearMade" and "Enclosure": 0.624195267808


In [9]:
'''
Add in a third feature "ProductGroup", again after dummified.
'''
dfenclo = dfenclo.dropna(subset = ['ProductGroup'])
dummies2 = pd.get_dummies(dfenclo['ProductGroup'])
dfpro = pd.concat([dfenclo, dummies2], axis = 1)
dfpro.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Steering_Controls,EROPS,EROPS w AC,OROPS,BL,MG,SSL,TEX,TTT,WL
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,11/16/2006 0:00,...,Conventional,0,1,0,0,0,0,0,0,1
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,3/26/2004 0:00,...,Conventional,0,1,0,0,0,0,0,0,1
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2/26/2004 0:00,...,NaN,0,0,1,0,0,1,0,0,0
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,5/19/2011 0:00,...,NaN,0,1,0,0,0,0,1,0,0
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,7/23/2009 0:00,...,NaN,1,0,0,0,0,1,0,0,0


In [10]:
y = dfpro['SalePrice'].values.reshape(-1, 1)
X = dfpro[['EROPS', 'EROPS w AC', 'BL', 'MG', 'SSL', 'TEX', 'TTT']].values.reshape(-1, 7)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

linear = LinearRegression()
linear.fit(X_train, y_train)

test_predicted = linear.predict(X_test)
print 'RMSLE with 3 features "YearMade", "Enclosure" and "ProductGroup":', score2(y_test, test_predicted)
#best model so far!

RMSLE with 3 features "YearMade", "Enclosure" and "ProductGroup": 0.564482887995


In [11]:
'''
Now use SVM and kNN instead of linear regression to build models and compare with current model.
'''
knn = KNeighborsRegressor()
knn.fit(X_train, y_train)

test_predicted = knn.predict(X_test)
print 'RMSLE with kNN:', score2(y_test, test_predicted)

svm = SVR()
svm.fit(X_train, y_train)

test_predicted = svm.predict(X_test)
print 'RMSLE with SVM:', score2(y_test, test_predicted)
#both perform worse than linear regression model

RMSLE with kNN: 0.735664392596
RMSLE with SVM: 0.658729478374


In [12]:
'''
Make prediction with the test data using the best model (linear regression) with 3 features.
'''
dftest = pd.read_csv('data/test.csv')
dftest.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,...,None or Unspecified,"12' 4""",None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN
2,1222841,999308,4587,121,3,2000,10127.0,Medium,1/5/2012 0:00,270LC,...,None or Unspecified,"12' 4""",None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1222843,1062425,1954,121,3,1000,4682.0,Low,1/5/2012 0:00,892DLC,...,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,1/4/2012 0:00,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional


In [13]:
dummies = pd.get_dummies(dftest['Enclosure'])
dftest = pd.concat([dftest, dummies], axis = 1)
dftest = dftest.drop(['EROPS AC'], axis = 1)
dummies2 = pd.get_dummies(dftest['ProductGroup'])
dftest = pd.concat([dftest, dummies2], axis = 1)

In [14]:
X_test = dftest[['EROPS', 'EROPS w AC', 'BL', 'MG', 'SSL', 'TEX', 'TTT']].values.reshape(-1, 7)

test_predicted = linear.predict(X_test)

In [15]:
len(test_predicted)

11573

In [16]:
test_predicted[:10]

array([[ 28070.19722179],
       [ 49935.0513972 ],
       [ 49935.0513972 ],
       [ 28070.19722179],
       [ 52351.61951209],
       [  9612.10650684],
       [ 30486.76533668],
       [ 30486.76533668],
       [ 38699.99902655],
       [ 32050.32058832]])